In [1]:
import pandas as pd
from numpy import int64
from datetime import date
from datetime import datetime

In [2]:
EST_MATRICULADOS = pd.read_excel('./INPUT/EstudiantesMatriculadosNiv.xlsx', engine = 'openpyxl')
VERSIONES_NIV = ['./INPUT/2022-1-2499-NP1-Presencial-Examen de nivelación_-calificaciones.csv', './INPUT/2022-1-2499-NP1-Presencial-Examen de nivelación-calificaciones.csv']

In [3]:
CALIFICACIONES = []
for cal in VERSIONES_NIV:
    CALIFICACIONES.append(pd.read_csv(cal, sep = ',')[:-1])
CALIFICACIONES = pd.concat(CALIFICACIONES, ignore_index = True)

In [4]:
aprobados = CALIFICACIONES[CALIFICACIONES['Calificación/5.00'] >= 3]

In [5]:
cedulas = []
for index, row in aprobados.iterrows():
    cedulas.append(EST_MATRICULADOS[EST_MATRICULADOS['Nombre de usuario'] == row['Nombre de usuario']]['Cédula'].iloc[0])
aprobados['Cédula'] = cedulas

C:\Users\radou\AppData\Local\Temp/ipykernel_11016/423137488.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aprobados['Cédula'] = cedulas


In [6]:
porNivelar = pd.merge(EST_MATRICULADOS, aprobados, on=['Nombre de usuario'], how="outer", indicator=True
              ).query('_merge=="left_only"')
porNivelar = porNivelar[['Nombre_x', 'Apellido(s)_x', 'Nombre de usuario','Cédula_x','Número de ID_x']]

In [7]:
porNivelar.rename(columns = {
    'Nombre_x' : 'Nombres',
    'Apellido(s)_x' : 'Apellidos',
    'Nombre de usuario' : 'Código UIS',
    'Cédula_x' : 'Cédula',
    'Número de ID_x' : 'ID Moodle'
}, inplace = True)

In [8]:
aprobados = aprobados[['Nombre', 'Apellido(s)', 'Nombre de usuario', 'Cédula', 'Número de ID', 'Comenzado el', 'Finalizado', 'Tiempo requerido', 'Calificación/5.00']]
aprobados.rename(columns = {
    'Nombre' : 'Nombres',
    'Apellido(s)': 'Apellidos',
    'Nombre de usuario' : 'Código UIS',
    'Número de ID': 'ID Moodle',
}, inplace = True)
aprobados['Código UIS'] = aprobados['Código UIS'].astype(int64, errors='ignore')

In [9]:
nombreArchivo = 'INFORME_NIVELACIÓN_' + str(date.today()) + '_Hora-' + str(datetime.now().hour) + '.xlsx'
writer = pd.ExcelWriter('./OUTPUT/' + nombreArchivo, engine='openpyxl')
porNivelar.to_excel(writer, sheet_name = 'Estudiantes pendientes nivelar', index = False)    
aprobados.to_excel(writer, sheet_name = 'Estudiantes que aprobaron', index = False)  
writer.close()